### Notebook - Data scrapping of players

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from dotenv import load_dotenv
import os

##### 1. User setting

In this first part, we will obtain from Riot API, the user PUUID which is the main parameter to obtain in order to get all the important information (kills, gold, cs, etc..)

In [71]:
# API Key setting
load_dotenv("api.env")
API_KEY = os.getenv("API_KEY")
headers = {"X-Riot-Token": API_KEY}

# User setting
gameName = "Cotchanadé"
tagLine = "1234"
region = "europe"
user_url = f"https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"

# PUUID 
user_data = requests.get(user_url, headers=headers).json()
user_puuid = user_data['puuid']

##### 2. Statistics parsing

In [ ]:
# Games
match_number = 30 
match_list_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{user_puuid}/ids?start=0&count={match_number}&queue=420"
match_ids = requests.get(match_list_url, headers=headers).json()

i = 0
metrics = 8 # (KDA, CS/min, Gold/min, Damage/min, Damage taken/min, Vision Score, KP, Winrate  )
stat_data = np.zeros((metrics,match_number))

# Statistics Parsing Algorithm 
for match_id in match_ids:
    match_url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    match_data = requests.get(match_url, headers=headers).json()
    time_game = match_data['info']['gameDuration']/60
    
    for p in match_data['info']['participants']:
        if p['puuid'] == user_puuid:
            if p['deaths'] > 0 :
                stat_data[0,i] = (p['kills']+p['assists'])/p['deaths']  # KDA
            else :
                stat_data[0,i] = p['kills']+p['assists']
            
            stat_data[1,i] = (p['totalMinionsKilled']+p['neutralMinionsKilled'])/time_game  # CS/min
            stat_data[2,i] = p['goldEarned']/time_game  # Gold/min
            stat_data[3,i] = p['totalDamageDealtToChampions']/time_game  # Damage/min
            stat_data[4,i] = p['totalDamageTaken']/time_game # Damages taken
            stat_data[5,i] = p['visionScore']  # Vision Score
            stat_data[6,i] = p['challenges']['killParticipation'] # KP
            stat_data[7, i] = int(p['win'])
    i += 1
    time.sleep(1.5)  # Respecte le rate limit !

player_stat = []
player_stat.append(user_puuid)
for i in range (metrics):
    player_stat.append(round(np.mean(stat_data[i]),2))


##### 3. Data visualisation

In [73]:
rows = ['Player PUUID', 'KDA', 'CS/min', 'Gold/min', 'Damage/min', 'DamageTaken/min', 'Vision Score', 'Kill Participation', 'Win Rate']
df = pd.DataFrame(player_stat, index=rows, columns=["Valeur"])
df

,Valeur
Player PUUID,qyHoh15wLsj8O_4DK-aEONuWqi36NOk_zM2jWZql641zHH...
KDA,2.68
CS/min,7.11
Gold/min,415.44
Damage/min,657.12
DamageTaken/min,982.48
Vision Score,18.13
Kill Participation,0.38
Win Rate,0.47


##### 4. Scrapping players PUUID